### 환경 불러오기, 클라이언트 초기화

In [ ]:
# .env 로드(OPENAI_API_KEY 등)
import dotenv  # 환경변수 파일 로더
dotenv.load_dotenv()  # .env → os.environ

# OpenAI Python SDK 클라이언트
from openai import OpenAI  # REST 클라이언트
client = OpenAI()  # 기본 환경변수에서 키 로드

# 비동기/유틸
import asyncio  # 이벤트 루프 사용
import base64   # 이미지 base64 인코딩/디코딩

# Streamlit UI
import streamlit as st  # 웹 UI 프레임워크

# Agents SDK 구성요소
from agents import (
    Agent,              # 에이전트 정의
    Runner,             # 실행기(run / run_streamed)
    SQLiteSession,      # 대화 이력 저장소(SQLite)
    WebSearchTool,      # 호스티드 웹검색 도구
    # FileSearchTool,   # 파일검색(주석 처리 상태)
    ImageGenerationTool,# 이미지 생성 도구
    CodeInterpreterTool,# 코드 인터프리터 도구
    HostedMCPTool,      # 호스티드 MCP 도구
)
from agents.mcp.server import MCPServerStdio  # 로컬 MCP 서버 연결

# 외부 벡터스토어 ID(미리 생성해 둔 ID)
VECTOR_STORE_ID = "vs_68a0815f62388191a9c3701ceb237234"  # 업로드 파일 인덱싱 대상

# Streamlit 세션 상태에 SQLite 세션 보관(최초 1회만 생성)
if "session" not in st.session_state:  # 초기 구동 시
    st.session_state["session"] = SQLiteSession(
        "chat-history",               # 세션 식별자
        "chat-gpt-clone-memory.db",   # SQLite 파일 경로
    )
session = st.session_state["session"]  # 편의 참조


### 히스토리 페인딩
- session.get_items()로 과거 메시지 항목을 가져와 재렌더링
- 항목 유형별로 사용자 텍스트, 입력 이미지, 도구 호출 결과를 다르게 표시
- 이 함수만 동기 실행!

In [ ]:
# 과거 대화 이력을 Streamlit 채팅 UI로 재렌더링
async def paint_history():  # 비동기 함수(스트림/세션 호출)
    messages = await session.get_items()  # 히스토리 로드(list of dict)

    for message in messages:  # 메시지 항목 순회
        if "role" in message:  # user/assistant 메시지
            with st.chat_message(message["role"]):  # 역할별 말풍선
                if message["role"] == "user":
                    content = message["content"]  # 문자열 또는 이미지 파트 리스트
                    if isinstance(content, str):
                        st.write(content)  # 사용자 텍스트
                    elif isinstance(content, list):
                        for part in content:
                            if "image_url" in part:  # 데이터 URI 이미지
                                st.image(part["image_url"])
                else:  # assistant 계열
                    if message["type"] == "message":
                        # 모델 텍스트 출력(수식 $ 이스케이프)
                        st.write(message["content"][0]["text"].replace("$", "\\$"))

        # 도구 호출 등 시스템성 항목 분기
        if "type" in message:
            message_type = message["type"]
            if message_type == "web_search_call":
                with st.chat_message("ai"):
                    st.write("🔍 Searched the web...")
            elif message_type == "file_search_call":
                with st.chat_message("ai"):
                    st.write("🗂️ Searched your files...")
            elif message_type == "image_generation_call":
                image = base64.b64decode(message["result"])  # base64 → bytes
                with st.chat_message("ai"):
                    st.image(image)  # 생성 이미지
            elif message_type == "code_interpreter_call":
                with st.chat_message("ai"):
                    st.code(message["code"])  # 실행 코드 표시
            elif message_type == "mcp_list_tools":
                with st.chat_message("ai"):
                    st.write(f"Listed {message['server_label']}'s tools")
            elif message_type == "mcp_call":
                with st.chat_message("ai"):
                    st.write(
                        f"Called {message['server_label']}'s {message['name']} "
                        f"with args {message['arguments']}"
                    )

# 비동기 함수 1회 실행으로 히스토리 그리기
asyncio.run(paint_history())  # Jupyter가 아닌 Streamlit에서 정상 사용

### 상태 매핑
- 스트리밍 도중 세부 이벤트 타입을 UI 상태 배지로 매핑
- 검색, 코드, 이미지, MCP 등 각 도구의 진행상태를 라벨 + state로 업데이트

In [ ]:
# 스트리밍 이벤트 타입 → 상태 배지 매핑 및 업데이트
def update_status(status_container, event):  # status_container: st.status()
    status_messages = {
        "response.web_search_call.completed": ("✅ Web search completed.", "complete"),
        "response.web_search_call.in_progress": ("🔍 Starting web search...", "running"),
        "response.web_search_call.searching": ("🔍 Web search in progress...", "running"),

        "response.file_search_call.completed": ("✅ File search completed.", "complete"),
        "response.file_search_call.in_progress": ("🗂️ Starting file search...", "running"),
        "response.file_search_call.searching": ("🗂️ File search in progress...", "running"),

        "response.image_generation_call.generating": ("🎨 Drawing image...", "running"),
        "response.image_generation_call.in_progress": ("🎨 Drawing image...", "running"),

        "response.code_interpreter_call_code.done": ("🤖 Ran code.", "complete"),
        "response.code_interpreter_call.completed": ("🤖 Ran code.", "complete"),
        "response.code_interpreter_call.in_progress": ("🤖 Running code...", "complete"),
        "response.code_interpreter_call.interpreting": ("🤖 Running code...", "complete"),

        "response.mcp_call.completed": ("⚒️ Called MCP tool", "complete"),
        "response.mcp_call.failed": ("⚒️ Error calling MCP tool", "complete"),
        "response.mcp_call.in_progress": ("⚒️ Calling MCP tool...", "running"),

        "response.mcp_list_tools.completed": ("⚒️ Listed MCP tools", "complete"),
        "response.mcp_list_tools.failed": ("⚒️ Error listing MCP tools", "complete"),
        "response.mcp_list_tools.in_progress": ("⚒️ Listing MCP tools", "running"),

        "response.completed": (" ", "complete"),
    }

    if event in status_messages:              # 알려진 이벤트만 처리
        label, state = status_messages[event] # (라벨, 상태) 해석
        status_container.update(label=label, state=state)  # 배지 업데이트


### 에이전트 실행 - MCP 연결, 도구 세팅, streaming loop
- `MCPServerStdio`로 로컬 MCP 서버 2개 연결
    +  야후 파이낸스(`yfinance`), 시간 서버(`mcp-server-time`)
- Agent에 웹 검색, 이미지 생성, 코드 인터프리터, Hosted MCP 도구 연결
- `Runner.run_stream` -> `stream.stream_events()`루프로 응답 이벤트 처리
    + text delta, code cell delta, image partial event에 각각 UI 업데이트

In [ ]:
# 사용자가 입력을 보낼 때 호출되는 비동기 실행기
async def run_agent(message: str):  # message: 사용자가 보낸 텍스트
    # MCP 서버 1: 야후 파이낸스
    yfinance_server = MCPServerStdio(
        params={"command": "uvx", "args": ["mcp-yahoo-finance"]},  # uvx 이용 실행
        cache_tools_list=True,  # 툴 리스트 캐시
    )
    # MCP 서버 2: 타임존 서버
    timezone_server = MCPServerStdio(
        params={"command": "uvx", "args": ["mcp-server-time", "--local-timezone=America/New_York"]}
    )

    # 두 MCP 서버를 async 컨텍스트로 열고 닫음
    async with yfinance_server, timezone_server:

        # 사용자 대면 에이전트 정의
        agent = Agent(
            mcp_servers=[yfinance_server, timezone_server],  # 연결된 MCP 서버들
            name="ChatGPT Clone",                            # 에이전트 이름
            instructions="""                                # 역할/사용 규칙
        You are a helpful assistant.

        You have access to the followign tools:
            - Web Search Tool: Use this when the user asks a questions that isn't in your training data. Use this tool when the users asks about current or future events, when you think you don't know the answer, try searching for it in the web first.
            - File Search Tool: Use this tool when the user asks a question about facts related to themselves. Or when they ask questions about specific files.
            - Code Interpreter Tool: Use this tool when you need to write and run code to answer the user's question.
        """,
            tools=[
                WebSearchTool(),  # 웹 검색(호스티드)
                # FileSearchTool(   # 파일검색은 예시로 주석 처리
                #     vector_store_ids=[VECTOR_STORE_ID],
                #     max_num_results=3,
                # ),
                ImageGenerationTool(  # 이미지 생성 도구(품질/포맷 지정)
                    tool_config={
                        "type": "image_generation",
                        "quality": "high",
                        "output_format": "jpeg",
                        "partial_images": 1,  # 스트리밍 중간 이미지 표시 사용
                    }
                ),
                CodeInterpreterTool(  # 코드 인터프리터(격리 컨테이너 자동)
                    tool_config={
                        "type": "code_interpreter",
                        "container": {"type": "auto"},
                    }
                ),
                HostedMCPTool(  # 호스티드 MCP 엔드포인트
                    tool_config={
                        "server_url": "https://mcp.context7.com/mcp",
                        "type": "mcp",
                        "server_label": "Context7",
                        "server_description": "Use this to get the docs from software projects.",
                        "require_approval": "never",
                    }
                ),
            ],
        )

        # 어시스턴트 말풍선 컨테이너 + 진행 배지/출력 플레이스홀더 구성
        with st.chat_message("ai"):
            status_container = st.status("⏳", expanded=False)  # 진행 상태 배지
            code_placeholder = st.empty()   # 코드 인터프리터 코드 표시 영역
            image_placeholder = st.empty()  # 이미지 partial 표시 영역
            text_placeholder = st.empty()   # 일반 텍스트 표시 영역
            response = ""       # 텍스트 누적 버퍼
            code_response = ""  # 코드 셀 텍스트 누적 버퍼

            # 다른 블록에서 비우기 위해 세션상에 참조 저장
            st.session_state["code_placeholder"] = code_placeholder
            st.session_state["image_placeholder"] = image_placeholder
            st.session_state["text_placeholder"] = text_placeholder

            # 스트리밍 시작(세션 포함)
            stream = Runner.run_streamed(
                agent,
                message,
                session=session,  # SQLite 대화 맥락 재사용
            )

            # 원시 응답 이벤트 소비 루프
            async for event in stream.stream_events():
                if event.type == "raw_response_event":  # 하위 응답 이벤트 묶음
                    update_status(status_container, event.data.type)  # 상태 갱신

                    # 일반 텍스트 델타
                    if event.data.type == "response.output_text.delta":
                        response += event.data.delta                      # 누적
                        text_placeholder.write(response.replace("$", "\\$"))  # 표시(수식 이스케이프)

                    # 코드 인터프리터 코드 셀 델타
                    if event.data.type == "response.code_interpreter_call_code.delta":
                        code_response += event.data.delta  # 누적
                        code_placeholder.code(code_response)

                    # 이미지 partial 프레임
                    elif event.data.type == "response.image_generation_call.partial_image":
                        image = base64.b64decode(event.data.partial_image_b64)  # base64 → bytes
                        image_placeholder.image(image)  # 중간 이미지 미리보기


### 입력 프롬프트 처리, 파일 업로드(텍스트, 이미지)
- `st.chat_input(..., accept_file=True)`로 텍스트+파일 동시 수신
- 텍스트 파일은 OpenAI Files API로 업로드 후 Vector Store에 첨부
- 이미지 파일은 data URI로 session에 추가하여 히스토리에 남김
- 사용자 텍스트가 있으면 즉시 말풍선 렌더 후 run_agent 호출

In [ ]:
# 채팅 입력창(파일 허용: txt/jpg/jpeg/png)
prompt = st.chat_input(
    "Write a message for your assistant",   # 플레이스홀더
    accept_file=True,                       # 파일 허용
    file_type=["txt", "jpg", "jpeg", "png"] # 허용 확장자
)

if prompt:  # 사용자가 전송한 입력 존재
    # 이전 실행의 플레이스홀더 초기화(덮어쓰기 방지)
    if "code_placeholder" in st.session_state:
        st.session_state["code_placeholder"].empty()
    if "image_placeholder" in st.session_state:
        st.session_state["image_placeholder"].empty()
    if "text_placeholder" in st.session_state:
        st.session_state["text_placeholder"].empty()

    # 동봉 파일 처리
    for file in prompt.files:
        if file.type.startswith("text/"):  # 텍스트 파일
            with st.chat_message("ai"):
                with st.status("⏳ Uploading file...") as status:
                    # OpenAI Files API 업로드
                    uploaded_file = client.files.create(
                        file=(file.name, file.getvalue()),
                        purpose="user_data",
                    )
                    status.update(label="⏳ Attaching file...")
                    # 업로드한 파일을 벡터스토어에 첨부(향후 파일검색에 사용)
                    client.vector_stores.files.create(
                        vector_store_id=VECTOR_STORE_ID,
                        file_id=uploaded_file.id,
                    )
                    status.update(label="✅ File uploaded", state="complete")

        elif file.type.startswith("image/"):  # 이미지 파일
            with st.status("⏳ Uploading image...") as status:
                file_bytes = file.getvalue()  # 바이트 읽기
                base64_data = base64.b64encode(file_bytes).decode("utf-8")
                data_uri = f"data:{file.type};base64,{base64_data}"  # data URI 생성
                # 세션 히스토리에 사용자 입력 이미지로 추가
                asyncio.run(
                    session.add_items(
                        [
                            {
                                "role": "user",
                                "content": [
                                    {
                                        "type": "input_image",
                                        "detail": "auto",
                                        "image_url": data_uri,
                                    }
                                ],
                            }
                        ]
                    )
                )
                status.update(label="✅ Image uploaded", state="complete")
            with st.chat_message("human"):
                st.image(data_uri)  # 사용자가 올린 이미지 미리보기

    # 사용자 텍스트 처리
    if prompt.text:
        with st.chat_message("human"):
            st.write(prompt.text)          # 사용자 말풍선 찍기
        asyncio.run(run_agent(prompt.text))# 에이전트 실행(비동기 함수 → run)


### 사이드바(Streamlit UI) - 메모리 리셋, 히스토리 확인
- `“Reset memory”` 버튼으로 `session.clear_session()`
- 현재 세션의 저장 항목을 사이드바에 출력

In [ ]:
# 사이드바 유틸
with st.sidebar:
    reset = st.button("Reset memory")  # 메모리 리셋 버튼
    if reset:
        asyncio.run(session.clear_session())  # 세션 비우기
    # 현재 세션 항목 덤프(디버깅/시연용)
    st.write(asyncio.run(session.get_items()))
